# Net Position Analysis: Three-System Comparison

This notebook compares net position measurements from three independent systems:
1. **FFT Signal Processing** - High-precision frequency domain analysis
2. **Sonar Image Analysis** - Computer vision-based detection from multibeam sonar
3. **DVL Navigation** - Doppler velocity log-based positioning (reference)

All measurements are synchronized by timestamp and compared in:
- Distance measurements
- Pitch angle estimates
- XY position coordinates (net-relative frame)

## 1. Import Libraries and Configuration

In [1]:
from pathlib import Path
from utils.net_analysis import (
    load_sonar_analysis_results,
    load_navigation_dataset,
    load_fft_dataset,
    summarize_sonar_results,
    summarize_navigation_results,
    summarize_fft_results,
    synchronize_sonar_and_dvl,
    summarize_distance_alignment,
    prepare_three_system_comparison,
    summarize_xy_positions,
)

# Import config for verification
from utils.config import IMAGE_PROCESSING_CONFIG, TRACKING_CONFIG

# CONFIG VERIFICATION - Print exact values to ensure consistency
print("=== IMAGE_PROCESSING_CONFIG VALUES ===")
for key, value in IMAGE_PROCESSING_CONFIG.items():
    print(f"{key}: {value}")

print("\n=== TRACKING_CONFIG VALUES ===")
for key, value in TRACKING_CONFIG.items():
    print(f"{key}: {value}")

print("\nUtility modules ready!")

 CONFIG VALIDATION WARNING:
   Missing TRACKING_CONFIG keys: {'corridor_both_directions', 'corridor_widen', 'use_corridor_splitting'}
=== IMAGE_PROCESSING_CONFIG VALUES ===
binary_threshold: 128
use_advanced_momentum_merging: True
adaptive_angle_steps: 20
adaptive_base_radius: 3
adaptive_max_elongation: 1.0
momentum_boost: 10.0
adaptive_linearity_threshold: 0.75
downscale_factor: 2
top_k_bins: 8
min_coverage_percent: 0.3
gaussian_sigma: 5.0
basic_gaussian_kernel_size: 3
basic_gaussian_sigma: 1.0
basic_use_dilation: True
basic_dilation_kernel_size: 3
basic_dilation_iterations: 3
morph_close_kernel: 0
edge_dilation_iterations: 0
min_contour_area: 200
aoi_boost_factor: 10.0
max_distance_change_pixels: 20

=== TRACKING_CONFIG VALUES ===
use_elliptical_aoi: True
ellipse_expansion_factor: 0.5
aoi_boost_factor: 10.0
score_area_weight: 1.0
score_linearity_weight: 2.0
score_aspect_ratio_weight: 1.5
aoi_center_x_percent: 50
aoi_center_y_percent: 60
aoi_width_percent: 60
aoi_height_percent: 70
ce

## 2. Configuration and Setup

In [2]:
# Target bag to analyze
TARGET_BAG = "2024-08-20_17-02-00"  # Change this to your desired bag ID

# Data source paths
FFT_ROOT = Path("/Volumes/LaCie/SOLAQUA/relative_fft_pose")
PLOTS_DIR = Path("/Volumes/LaCie/SOLAQUA/exports/plots")
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Analyzing: {TARGET_BAG}")
print(f"Plots: {PLOTS_DIR}")
print(f"\nNOTE: To export data for this bag, use:")
print(f"python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data \\")
print(f"  --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem {TARGET_BAG} --all")

Analyzing: 2024-08-20_17-02-00
Plots: /Volumes/LaCie/SOLAQUA/exports/plots

NOTE: To export data for this bag, use:
python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data \
  --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem 2024-08-20_17-02-00 --all


## 3. Load Data from All Systems

In [3]:
# Load data from all three systems
print("=== LOADING DATA ===\n")

df_sonar, sonar_meta = load_sonar_analysis_results(TARGET_BAG)
df_nav, nav_meta = load_navigation_dataset(TARGET_BAG)
df_fft, fft_meta = load_fft_dataset(TARGET_BAG, fft_root=FFT_ROOT)

# Display data sources
print("Data Sources:")
for label, meta in {"Sonar": sonar_meta, "DVL": nav_meta, "FFT": fft_meta}.items():
    path = meta.get('resolved_path', 'Not found')
    rows = meta.get('rows', 0)
    status = "✓" if rows > 0 else "✗"
    print(f"  {status} {label}: {rows} records")
    if rows > 0:
        print(f"      {path}")

=== LOADING DATA ===

Data Sources:
  ✓ Sonar: 944 records
      /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_17-02-00_analysis.csv
  ✓ DVL: 497 records
      /Volumes/LaCie/SOLAQUA/exports/by_bag/navigation_plane_approximation__2024-08-20_17-02-00_data.csv
  ✓ FFT: 410 records
      /Volumes/LaCie/SOLAQUA/relative_fft_pose/2024-08-20_17-02-00_relative_pose_fft.csv


In [4]:
# Summaries for each dataset
print("\n=== DATA SUMMARIES ===\n")

summaries = {
    "Sonar": summarize_sonar_results(df_sonar),
    "DVL": summarize_navigation_results(df_nav),
    "FFT": summarize_fft_results(df_fft),
}

for label, stats in summaries.items():
    print(f"{label}:")
    for key, value in stats.items():
        if value is not None:
            if isinstance(value, tuple):
                print(f"  {key}: {value[0]:.3f} to {value[1]:.3f}")
            elif isinstance(value, float):
                print(f"  {key}: {value:.3f}")
            else:
                print(f"  {key}: {value}")
    print()


=== DATA SUMMARIES ===

Sonar:
  rows: 944
  detections: 944
  detection_rate: 100.000
  distance_range_m: 0.559 to 4.529
  angle_range_deg: 119.360 to 192.768

DVL:
  rows: 497
  distance_range_m: 0.430 to 3.340
  distance_mean_m: 1.121
  distance_std_m: 0.305
  pitch_range_deg: -60.513 to 85.257

FFT:
  rows: 410
  distance_range_m: -1.833 to 1.596
  distance_mean_m: 0.877
  distance_std_m: 0.320
  pitch_range_deg: -82.723 to 88.077



In [5]:
# Display sample data
if df_sonar is not None:
    print("Sonar Analysis Sample:")
    display_cols = ['timestamp', 'distance_meters', 'angle_degrees', 'detection_success']
    display_cols = [c for c in display_cols if c in df_sonar.columns]
    print(df_sonar[display_cols].head(5))
    print()

if df_nav is not None:
    print("DVL Navigation Sample:")
    display_cols = ['timestamp', 'NetDistance', 'NetPitch']
    display_cols = [c for c in display_cols if c in df_nav.columns]
    print(df_nav[display_cols].head(5))
    print()

if df_fft is not None:
    print("FFT Data Sample:")
    display_cols = ['timestamp', 'distance_m', 'pitch_deg']
    display_cols = [c for c in display_cols if c in df_fft.columns]
    print(df_fft[display_cols].head(5))

Sonar Analysis Sample:
                            timestamp  distance_meters  angle_degrees  \
0 2024-08-20 15:02:03.319382668+00:00         0.564078     172.282936   
1 2024-08-20 15:02:03.405480623+00:00         0.563349     172.212332   
2 2024-08-20 15:02:03.458926201+00:00         0.558668     172.252032   
3 2024-08-20 15:02:03.505110979+00:00         0.566952     172.122790   
4 2024-08-20 15:02:03.609708309+00:00         0.563451     172.115187   

   detection_success  
0               True  
1               True  
2               True  
3               True  
4               True  

DVL Navigation Sample:
                            timestamp  NetDistance  NetPitch
0 2024-08-20 15:02:05.408126593+00:00         0.72  -0.07037
1 2024-08-20 15:02:05.503421782+00:00         0.76  -0.09843
2 2024-08-20 15:02:05.603994131+00:00         0.77  -0.06585
3 2024-08-20 15:02:05.713511705+00:00         0.81  -0.07341
4 2024-08-20 15:02:05.810849667+00:00         0.82  -0.09939

FFT Data 

## 4. Two-System Synchronization (Sonar + DVL)

First, compare sonar and DVL systems independently.

In [6]:
# Synchronize sonar and DVL measurements
print("=== TWO-SYSTEM SYNCHRONIZATION (Sonar + DVL) ===\n")

sync_df = synchronize_sonar_and_dvl(df_sonar, df_nav, tolerance_seconds=1.0)
sync_summary = summarize_distance_alignment(sync_df)

print(f"Aligned pairs: {sync_summary['pairs']}")
print(f"Valid distance pairs: {sync_summary['valid_pairs']}")

if sync_summary["difference_stats"]:
    diff = sync_summary["difference_stats"]
    print(f"\nDistance Comparison (Sonar - DVL):")
    print(f"  Mean: {diff['mean']:.3f} m")
    print(f"  Std: {diff['std']:.3f} m")
    print(f"  Range: {diff['min']:.3f} to {diff['max']:.3f} m")
    print(f"  95th percentile: {diff['p95']:.3f} m")
    if 'correlation' in diff:
        print(f"  Correlation: {diff['correlation']:.3f}")
else:
    print("No overlapping distance measurements within tolerance.")

# Display sample synchronized data
if not sync_df.empty:
    print(f"\nSynchronized Data Sample:")
    display_cols = ['timestamp', 'sonar_distance_m', 'dvl_distance_m', 'sonar_angle_deg', 'dvl_pitch_deg', 'time_diff_s']
    display_cols = [c for c in display_cols if c in sync_df.columns]
    print(sync_df[display_cols].head(10))

=== TWO-SYSTEM SYNCHRONIZATION (Sonar + DVL) ===

Aligned pairs: 927
Valid distance pairs: 927

Distance Comparison (Sonar - DVL):
  Mean: 0.141 m
  Std: 0.662 m
  Range: -0.284 to 3.406 m
  95th percentile: 1.595 m
  Correlation: 0.472

Synchronized Data Sample:
                            timestamp  sonar_distance_m  dvl_distance_m  \
0 2024-08-20 15:02:04.418097496+00:00          0.568634            0.72   
1 2024-08-20 15:02:04.460487843+00:00          0.568740            0.72   
2 2024-08-20 15:02:04.520919323+00:00          0.570406            0.72   
3 2024-08-20 15:02:04.586270809+00:00          0.568945            0.72   
4 2024-08-20 15:02:04.647171021+00:00          0.569883            0.72   
5 2024-08-20 15:02:04.712663174+00:00          0.567835            0.72   
6 2024-08-20 15:02:04.801765203+00:00          0.566873            0.72   
7 2024-08-20 15:02:04.849609375+00:00          0.570798            0.72   
8 2024-08-20 15:02:04.905240774+00:00          0.568614      

/Users/eirikvarnes/code/SOLAQUA/utils/net_analysis.py:210: UserWarning: no explicit representation of timezones available for np.datetime64
  sonar_ts = np.datetime64(sonar_row.timestamp)


## 5. Three-System Comparison

Synchronize and compare all three systems (FFT + Sonar + DVL).

In [7]:
# Perform multi-system comparison (works with 2 or 3 systems)
print("\n=== MULTI-SYSTEM COMPARISON ===\n")

# Determine which systems are available
available_systems = []
if df_sonar is not None and not df_sonar.empty:
    available_systems.append("Sonar")
if df_nav is not None and not df_nav.empty:
    available_systems.append("DVL")
if df_fft is not None and not df_fft.empty:
    available_systems.append("FFT")

print(f"Available systems: {', '.join(available_systems)}")

if len(available_systems) < 2:
    print("✗ Need at least 2 systems for comparison")
    three_sync_df = None
    figs = {}
    visualizer = None
else:
    # Run multi-system comparison
    system_count = len(available_systems)
    print(f"Running {system_count}-system comparison...")
    
    three_sync_df, figs, visualizer = prepare_three_system_comparison(
        TARGET_BAG,
        df_sonar,
        df_nav,
        df_fft,
        tolerance_seconds=0.5
    )
    
    if three_sync_df is None or three_sync_df.empty:
        print("✗ No overlapping timestamps across available systems")
        three_sync_df = None
    else:
        print(f"✓ Synchronized {len(three_sync_df)} records across {system_count} systems\n")
        
        # DEBUG: Check what columns we have
        print("Available columns:")
        print(f"  {list(three_sync_df.columns)}")
        
        # Check for distance and pitch columns
        distance_cols = [c for c in three_sync_df.columns if 'distance' in c.lower()]
        pitch_cols = [c for c in three_sync_df.columns if 'pitch' in c.lower() or 'angle' in c.lower()]
        
        print(f"\nDistance columns: {distance_cols}")
        print(f"Pitch columns: {pitch_cols}")
        
        # Check if XY columns already exist (with _m suffix)
        existing_xy = [c for c in three_sync_df.columns if ('_x_m' in c or '_y_m' in c or c.endswith('_x') or c.endswith('_y'))]
        
        if existing_xy:
            print(f"\n✓ XY coordinates already exist: {existing_xy}")
            
            # Rename _m suffix columns to match visualizer expectations
            rename_map = {}
            if 'fft_x_m' in three_sync_df.columns:
                rename_map['fft_x_m'] = 'fft_x'
                rename_map['fft_y_m'] = 'fft_y'
            if 'sonar_x_m' in three_sync_df.columns:
                rename_map['sonar_x_m'] = 'sonar_x'
                rename_map['sonar_y_m'] = 'sonar_y'
            if 'nav_x_m' in three_sync_df.columns:
                rename_map['nav_x_m'] = 'dvl_x'  # Note: nav → dvl
                rename_map['nav_y_m'] = 'dvl_y'
            
            if rename_map:
                three_sync_df = three_sync_df.rename(columns=rename_map)
                print(f"   Renamed columns for visualizer: {list(rename_map.keys())} → {list(rename_map.values())}")
        else:
            # Compute XY if they don't exist at all
            print("\n⚠️  XY coordinates not found, attempting to compute...")
            
            import numpy as np
            
            # Compute XY for each system that has distance and pitch
            if 'sonar_distance_m' in three_sync_df.columns and 'sonar_pitch_deg' in three_sync_df.columns:
                print("  Computing Sonar XY...")
                three_sync_df['sonar_x'] = three_sync_df['sonar_distance_m'] * np.sin(np.radians(three_sync_df['sonar_pitch_deg']))
                three_sync_df['sonar_y'] = three_sync_df['sonar_distance_m'] * np.cos(np.radians(three_sync_df['sonar_pitch_deg']))
            
            if 'nav_distance_m' in three_sync_df.columns and 'nav_pitch_deg' in three_sync_df.columns:
                print("  Computing DVL XY...")
                three_sync_df['dvl_x'] = three_sync_df['nav_distance_m'] * np.sin(np.radians(three_sync_df['nav_pitch_deg']))
                three_sync_df['dvl_y'] = three_sync_df['nav_distance_m'] * np.cos(np.radians(three_sync_df['nav_pitch_deg']))
            
            if 'fft_distance_m' in three_sync_df.columns and 'fft_pitch_deg' in three_sync_df.columns:
                print("  Computing FFT XY...")
                three_sync_df['fft_x'] = three_sync_df['fft_distance_m'] * np.sin(np.radians(three_sync_df['fft_pitch_deg']))
                three_sync_df['fft_y'] = three_sync_df['fft_distance_m'] * np.cos(np.radians(three_sync_df['fft_pitch_deg']))
            
            # Report what was computed
            xy_cols = [c for c in three_sync_df.columns if c.endswith('_x') or c.endswith('_y')]
            print(f"  ✓ Computed XY columns: {xy_cols}")


=== MULTI-SYSTEM COMPARISON ===

Available systems: Sonar, DVL, FFT
Running 3-system comparison...
✓ Synchronized 1851 records across 3 systems

Available columns:
  ['sync_timestamp', 'fft_distance_m', 'fft_pitch_deg', 'fft_x_m', 'fft_y_m', 'fft_heading', 'sonar_distance_m', 'sonar_pitch_deg', 'sonar_x_m', 'sonar_y_m', 'sonar_detection_success', 'nav_distance_m', 'nav_pitch_deg', 'nav_x_m', 'nav_y_m']

Distance columns: ['fft_distance_m', 'sonar_distance_m', 'nav_distance_m']
Pitch columns: ['fft_pitch_deg', 'sonar_pitch_deg', 'nav_pitch_deg']

✓ XY coordinates already exist: ['fft_x_m', 'fft_y_m', 'sonar_x_m', 'sonar_y_m', 'nav_x_m', 'nav_y_m']
   Renamed columns for visualizer: ['fft_x_m', 'fft_y_m', 'sonar_x_m', 'sonar_y_m', 'nav_x_m', 'nav_y_m'] → ['fft_x', 'fft_y', 'sonar_x', 'sonar_y', 'dvl_x', 'dvl_y']
✓ Synchronized 1851 records across 3 systems

Available columns:
  ['sync_timestamp', 'fft_distance_m', 'fft_pitch_deg', 'fft_x_m', 'fft_y_m', 'fft_heading', 'sonar_distance_m',

## 6. Distance and Pitch Comparison Plots

In [8]:
# Display comparison plots
if three_sync_df is not None and not three_sync_df.empty and figs:
    print("=== DISPLAYING COMPARISON PLOTS ===\n")
    
    # Display all generated plots
    for name, fig in figs.items():
        if fig is not None:
            print(f"Displaying {name.replace('_', ' ').title()}...")
            try:
                fig.show()
                
                # Save plots
                save_path = PLOTS_DIR / f"{TARGET_BAG}_{name}.html"
                fig.write_html(str(save_path))
                print(f"  ✓ Saved: {save_path.name}\n")
            except Exception as e:
                print(f"  ✗ Warning: Could not display/save - {e}\n")
    
    print("✓ All plots displayed and saved")
else:
    print("✗ No synchronized data or plots available")
    print("  This usually means:")
    print("  - Timestamps don't overlap between systems")
    print("  - Data quality issues")
    print("  - Missing required columns in datasets")

=== DISPLAYING COMPARISON PLOTS ===

Displaying Distance Comparison...


  ✓ Saved: 2024-08-20_17-02-00_distance_comparison.html

Displaying Pitch Comparison...


  ✓ Saved: 2024-08-20_17-02-00_pitch_comparison.html

✓ All plots displayed and saved


## 7. Net-Relative XY Position Plots

Visualize the robot's trajectory relative to the net plane in XY coordinates from all systems.

In [9]:
# Display XY position plots
if three_sync_df is not None and not three_sync_df.empty and visualizer is not None:
    print("=== DISPLAYING XY POSITION PLOTS ===\n")
    
    # Check which XY columns are available
    has_sonar_xy = 'sonar_x' in three_sync_df.columns and 'sonar_y' in three_sync_df.columns
    has_dvl_xy = 'dvl_x' in three_sync_df.columns and 'dvl_y' in three_sync_df.columns
    has_fft_xy = 'fft_x' in three_sync_df.columns and 'fft_y' in three_sync_df.columns
    
    available_xy_systems = []
    if has_sonar_xy:
        available_xy_systems.append("Sonar")
    if has_dvl_xy:
        available_xy_systems.append("DVL")
    if has_fft_xy:
        available_xy_systems.append("FFT")
    
    print(f"Systems with XY coordinates: {', '.join(available_xy_systems)}")
    
    if len(available_xy_systems) == 0:
        print("✗ No XY position data available")
        print("  XY coordinates require distance and pitch measurements")
    else:
        # Configure lateral speed (robot moving along net)
        LATERAL_SPEED_M_S = 0.2  # meters per second (adjust based on actual speed)
        
        # Generate 3D XY trajectory plot
        print(f"\nGenerating 3D XY trajectory plot...")
        print(f"  Lateral speed: {LATERAL_SPEED_M_S} m/s")
        try:
            fig_xy = visualizer.plot_xy_trajectories(three_sync_df, lateral_speed_m_s=LATERAL_SPEED_M_S)
            
            if fig_xy is not None:
                # Display the plot inline
                display(fig_xy)
                
                # Save plot
                save_path = PLOTS_DIR / f"{TARGET_BAG}_xy_trajectories_3d.html"
                fig_xy.write_html(str(save_path))
                print(f"✓ Saved: {save_path.name}\n")
            else:
                print("✗ Could not generate XY trajectory plot\n")
                
        except Exception as e:
            print(f"✗ Error generating XY plot: {e}\n")
            import traceback
            traceback.print_exc()
        
        # Generate individual XY component comparison plots if multiple systems available
        if len(available_xy_systems) >= 2:
            print("\nGenerating XY component comparison plots...")
            try:
                # X position comparison (perpendicular distance to net)
                print("\n--- Perpendicular Distance to Net Over Time ---")
                fig_x = visualizer.plot_xy_component_comparison(three_sync_df, component='x')
                if fig_x is not None:
                    display(fig_x)
                    save_path = PLOTS_DIR / f"{TARGET_BAG}_x_position_comparison.html"
                    fig_x.write_html(str(save_path))
                    print(f"✓ X position comparison saved: {save_path.name}\n")
                
                # Y position comparison (lateral position along net)
                print("--- Lateral Position Along Net Over Time ---")
                fig_y = visualizer.plot_xy_component_comparison(three_sync_df, component='y')
                if fig_y is not None:
                    display(fig_y)
                    save_path = PLOTS_DIR / f"{TARGET_BAG}_y_position_comparison.html"
                    fig_y.write_html(str(save_path))
                    print(f"✓ Y position comparison saved: {save_path.name}\n")
                
            except Exception as e:
                print(f"✗ Error generating component plots: {e}")
                import traceback
                traceback.print_exc()
        
        print("✓ XY position plots complete")
        
else:
    print("✗ No synchronized data or visualizer available for XY plots")

=== DISPLAYING XY POSITION PLOTS ===

Systems with XY coordinates: Sonar, DVL, FFT

Generating 3D XY trajectory plot...
  Lateral speed: 0.2 m/s


✓ Saved: 2024-08-20_17-02-00_xy_trajectories_3d.html


Generating XY component comparison plots...

--- Perpendicular Distance to Net Over Time ---


✓ X position comparison saved: 2024-08-20_17-02-00_x_position_comparison.html

--- Lateral Position Along Net Over Time ---


✓ Y position comparison saved: 2024-08-20_17-02-00_y_position_comparison.html

✓ XY position plots complete


## 8. Position Statistics and Analysis

In [10]:
# Display detailed position statistics
if three_sync_df is not None and not three_sync_df.empty:
    print("=== POSITION STATISTICS ===\n")
    
    # XY Position Summary
    xy_stats = summarize_xy_positions(three_sync_df)
    
    if xy_stats:
        for system, stats in xy_stats.items():
            print(f"{system}:")
            print(f"  Data points: {stats['count']}")
            print(f"  X position: {stats['x_mean']:.3f} ± {stats['x_std']:.3f} m")
            print(f"  Y position: {stats['y_mean']:.3f} ± {stats['y_std']:.3f} m")
            print(f"  Distance from origin: {stats['distance_mean']:.3f} ± {stats['distance_std']:.3f} m")
            print()
    else:
        print("No XY position data available")
    
    # Additional analysis if visualizer available
    if visualizer is not None:
        try:
            print("=== DETAILED POSITION ANALYSIS ===\n")
            visualizer.print_position_summary(three_sync_df)
        except Exception as e:
            print(f"Could not print detailed analysis: {e}")
else:
    print("✗ No synchronized data available for position analysis")

=== POSITION STATISTICS ===

FFT:
  Data points: 1769
  X position: 0.820 ± 0.379 m
  Y position: 0.039 ± 0.122 m
  Distance from origin: 0.849 ± 0.335 m

Sonar:
  Data points: 1851
  X position: 1.089 ± 0.328 m
  Y position: -0.212 ± 0.595 m
  Distance from origin: 1.162 ± 0.586 m

DVL:
  Data points: 1813
  X position: 1.097 ± 0.293 m
  Y position: -0.080 ± 0.285 m
  Distance from origin: 1.126 ± 0.332 m

=== DETAILED POSITION ANALYSIS ===

Position data summary:


## 9. Summary

In [11]:
print("\n" + "="*70)
print("ANALYSIS SUMMARY")
print("="*70)

print(f"\nTarget Bag: {TARGET_BAG}")

print(f"\nData Availability:")
for label, meta in {"Sonar": sonar_meta, "DVL": nav_meta, "FFT": fft_meta}.items():
    rows = meta.get('rows', 0)
    status = "✓" if rows > 0 else "✗"
    print(f"  {status} {label}: {rows} records")

print(f"\nSynchronization Results:")
print(f"  Two-system (Sonar+DVL): {len(sync_df)} records")
if three_sync_df is not None:
    print(f"  Multi-system: {len(three_sync_df)} records")
else:
    print(f"  Multi-system: N/A")

if three_sync_df is not None and not three_sync_df.empty:
    # Count available plots
    plot_count = len(figs) if figs else 0
    
    # Check for XY plots
    has_xy = ('sonar_x' in three_sync_df.columns or 
              'dvl_x' in three_sync_df.columns or 
              'fft_x' in three_sync_df.columns)
    
    if has_xy:
        plot_count += 3  # XY trajectory + X comparison + Y comparison
    
    print(f"\nGenerated Plots: {plot_count}")
    if figs:
        for name in figs.keys():
            print(f"  - {name.replace('_', ' ').title()}")
    if has_xy:
        print(f"  - XY Trajectories")
        print(f"  - X Position Comparison")
        print(f"  - Y Position Comparison")
    
    print(f"\nSaved to: {PLOTS_DIR}")

print("\n✓ Notebook complete")


ANALYSIS SUMMARY

Target Bag: 2024-08-20_17-02-00

Data Availability:
  ✓ Sonar: 944 records
  ✓ DVL: 497 records
  ✓ FFT: 410 records

Synchronization Results:
  Two-system (Sonar+DVL): 927 records
  Multi-system: 1851 records

Generated Plots: 5
  - Distance Comparison
  - Pitch Comparison
  - XY Trajectories
  - X Position Comparison
  - Y Position Comparison

Saved to: /Volumes/LaCie/SOLAQUA/exports/plots

✓ Notebook complete
